# Exploration of College Brook C-Q Dynamics and Hysteresis

This project explores nitrate C-Q relationships and hysteresis with high-frequency data from two stations in College Brook, NH.

### Introduction

An excess of nutrients is one of the major threats to aquatic ecosystems in today’s human-impacted watersheds. Around the mid-20th century, The Green Revolution revolutionized agricultural practices, sustaining a global “population explosion,” but inadvertently shifting natural, balanced nitrogen cycling. The widespread use of nitrogen-based fertilizers and nitrogen-fixing crops, along with the increased burning of fossil fuels, has substantially raised reactive nitrogen levels on a global scale. Nitrate is an essential nutrient for terrestrial and aquatic ecosystems, but when its supply exceeds biological demand, it can trigger a cascade of harmful effects—such as eutrophication, biodiversity loss, and acidification—as well as broader impacts on atmosphere condition, air quality, and human health (Smith 1999, Galloway 2003). Aquatic ecosystem function is the primary source of nitrogen retention and removal within the nitrogen cycle, and is therefore viewed as the primary opportunity for intervention. Effective mitigation of nitrogen impairment begins with a robust understanding of river network nutrient inputs, in-stream dynamics, and total flux.

The relationships between stream discharge and reactive nutrient concentrations such as nitrate are highly variable and dynamic. Discharge-based total flux estimates that assume a constant concentration quickly breakdown—thus underscoring the need for higher resolution nutrient studies. Concentration-discharge plots are frequently used to examine these relationships, across time, space, and other variables. The relationship between nitrate concentration and discharge can help to understand of active source areas (proximity and supply) and transport flowpaths (speed, length, relative subsurface depth) (Evans and Davies 1998, Zuecco et al. 2016). C-Q relationships that do not match on the rising versus falling limbs of a storm/discharge peak—referred to as hysteresis—often indicate useful information such as an exhaustible nutrient supply, mobilization/activation of a different source area, or changes in nutrient uptake and retention.

Storm events dominate total nutrient flux through river networks, compared to baseflows, and are therefore particularly important for understanding broader nutrient dynamics. Storm flows also often display the most complex, impactful C-Q patterns and hysteresis. The frequency and intensity of storms is expected to increase across the world (Whitehead et al. 2009), making intra-storm nutrient responses doubly important. Yet, there are relatively few studies on hydrologic and nutrient responses to storm events due to the inherent difficulty of capturing sufficiently high-resolution data of such dynamic, ephemeral events. High-frequency, in situ sensors are revolutionizing our ability to gain insight into the nutrient dynamics of storm events.

### Research questions and hypotheses

**Two central research questions** will guide this prjoect:

1. What influence do storm characteristics (seasonality, intensity, duration, frequency/occurrence interval, and antecedent conditions) have on nutrient sources, transport, and total loading in river networks? 

Knowledge of these hydroclimatic influences is essential for improving predictive models and understanding the possible effects of a changing precipitation regime on water quality management.

2. What influence does land-use have on these patterns of storm response nutrient export, on a sub-catchment scale?

Nutrient sources and transport pathways are often highly heterogeneous across a patchwork of land uses throughout a watershed, leading to high variability of inputs. Assessing the impact of land-use on storm response nutrient loading at a sub-catchment, reach-level granularity is novel, and can improve our understanding of the specific scale at which nutrient loads vary and can be addressed with targeted management and restoration practices.

**Hypotheses**

1. Storm characteristics **will** have an effect and **significant correlation** with patterns of nutrient storm response. A buildup of nutrients over dry spells will lead to "flushing" during the next storm event, delivering higher concentrations to the stream, especially with a big initial pulse. Seasonoally this will be most evident in fall storms after the drier summer monthes. Meanwhile, spring storm, being more frequent, will not exihibit the same "flush" of built-up nutrients, and rather will be "diluting" with relatively lower nutrient fluxes.

2. More intense and/or prolonged storms may activate a more distal source area and/or slower flow pathways than smaller storms, and thus may exhibit a secondary signature of nutrient delivery later in the event (on the falling limb).

3. More forested sections of the catchment will display the most mediated mutrient loading, with lower concentrations and less hysteresis, due to uptake by plants, deep soils, and lack of human-impacts/nutrient sources. Urban reaches feature high proportions of impervious surfaces, as well as nutrient additions (such as lawn fetilizers), and thus will deliver very rapid initial flushes of nutrients, before transitioning to a "diluting" force. Agricultural areas will offer the most inexhaustable nutrient source due to manure and/or fertilizer applications, but may have a complex hysteresis response depending on how proximal the highest concentrations on nutrients are to the stream. 

### Study area

The study site for these analyses is College Brook -- a small headwater stream residing in a mixed land-use catchment in Durham, New Hampshire. Withing College Brook, there are two study sites. The first, called CLBG.AG, is in an intermittent reach just downstream from an agricultural area (The UNH Fairchild Dairy Teaching and Research Center). The second, CLGB.UPPER is downstream just 1/4 mile or so, with College Woods bordering the riverbank right between the two, and athletic fields and some parking lots on river-left. Critically, it is between these two stations that a restoration has been proposed, to restore stream function and buffer stormwater flows and nitrate pulses.

Figure 1. Site map

### Data description

### Setup

In [ ]:
"""

Parameters
----------
q_files : list of strings of discharge (Q) datafiles. i.e. ['CLGBag_Q_2022-2025.csv']
n_files : list of strings of discharge (Q) datafiles. i.e. ['CLGBag_N_2022-2023.csv', 'CLGBag_N_2024.csv']

@author: josephbaldus
@date = 2025-10-20
@license = MIT -- https://opensource.org/licenses/MIT

"""

In [ ]:
#%% Imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
#%% Specified parameters to change!

q_files = ['CLGBag_Q_2022-2025.csv']
n_files = ['CLGBag_N_2022-2023.csv', 'CLGBag_N_2024.csv']

In [ ]:
#%% GLOBAL VARIABLE/UNIT CONVERSIONS


### Prepare data

This section reads in all the high-frequency discharge and nitrate datafiles from the in-situ optical nitrate analyzer sensors and pressure transducers. We then do some data cleaning, leaving out erroneous readings, and merge all data into one dataset for further analysis.

In [ ]:
#%% Load data function

def load(filename):
    df = pd.read_csv(filename, index_col=[0])
    df.index = pd.to_datetime(df.index, format='mixed', errors='coerce')
    df.columns = df.columns.str.lower().str.strip()
    return df

In [ ]:
#%% Load all N file(s)

raw_ndata = pd.DataFrame()

for file in n_files:
    raw_ndata = pd.concat([raw_ndata, load(file)], axis = 0)
    
# N QC flag columns
if 'no3.mgl.qf' in raw_ndata.columns.tolist():
    n_data = raw_ndata[raw_ndata['no3.mgl.qf'] == False] #false is good

if 'flag' in raw_ndata.columns.tolist():
    n_data = raw_ndata[~raw_ndata['flag'].isin([1,2])] #1,2 is bad


In [ ]:
#%% Load Q file(s)

raw_qdata = pd.DataFrame()

for file in q_files:
    raw_qdata = pd.concat([raw_qdata, load(file)], axis = 0)
    
# Q QC flag columns
if 'q.m3sqf' in raw_qdata.columns.tolist():
    q_data = raw_qdata[raw_qdata['q.m3sqf'] == False]

In [ ]:
#%% Combine N and Q files

keep_cols = ['no3.mgl', 'q.m3s', 'measured.q.m3s']

data = pd.merge(n_data, q_data, how='outer', left_index=True, right_index=True)
data = data[keep_cols]

data.rename(columns={'no3.mgl': 'N'}, inplace=True)
data.rename(columns={'q.m3s': 'Q'}, inplace=True)

In [ ]:
#%% Checks function

check_n = n_data[n_data['no3.mgl'].notna()]
check_n2 = data[data['N'].notna()]

check_q = q_data[q_data['q.m3s'].notna()]
check_q2 = data[data['Q'].notna()]


### Time-series plots

This section creates plots of discharge and nitrate concentrations over time. First over the whole study area, from 2022 to 2024, and then zooming in on several individual storm events that we will continue to investigate in future analyses.

In [ ]:
# Write time series plotting function?

In [ ]:
# Plots!

#data = data.loc['2024-08-01':'2024-08-15'] #to subset data/zoom in, will integrate into a plot function

fig1, ax1 = plt.subplots(figsize=(12, 4))

# Plot nitrate concentration (left Y-axis)
ax1.plot(data.index, data.N, color='orange', label="Nitrate (mg/L)", linewidth=1)
ax1.set_ylabel("Nitrate concentration (mg/L)", color='orange')
ax1.tick_params(axis='y', labelcolor='orange')

# Create a twin axis sharing the same X-axis
ax2 = ax1.twinx()

# Plot discharge (right Y-axis)
ax2.plot(data.index, data.Q, color='blue', label="Discharge (m³/s)", linewidth=.5)
ax2.set_ylabel("Discharge (m³/s)", color='blue')
ax2.tick_params(axis='y', labelcolor='blue')

# Add title
ax1.set_title("College Brook, NH: Q versus N over time")

# Auto-format date labels
fig1.autofmt_xdate()

# Combine legends from both axes
lines_1, labels_1 = ax1.get_legend_handles_labels()
lines_2, labels_2 = ax2.get_legend_handles_labels()
ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='upper left')

plt.show()

In [ ]:
# Layer in precipitation record to show delay from precip to Q to C?

In [ ]:
# Select a few storms to zoom in on, still time series plotting.

### C-Q plots

In [1]:
# First, plot all C-values against Q-values to examine general trend/relationship.
# 3 ways: Linear, log-linear, and log-log.

In [ ]:
# Turn whicher one best fits relationship into a function

In [ ]:
# To then plot individual storm events on C-Q, showing time progression + hysteresis. (Select and demarcate storms manually)

In [ ]:
# Compare storms from different seasons, between two sites (to show land-use effects), and of different sized

### Calculate Hysteresis Indicies to compare quantitatively/objectively (if time allows and ambitions are high, otherwise for the future)

Involves normalizing C-Q plot data, calculating integrals along rising versus falling limbs, then subtracting each pair of integrals where Q is the same between rising and falling. The sum of those integral differences gives a hysteresis index value h. Where h is positive, indicates clockwise hysteresis. Where negative, hysteresis is anticlockwise.

### Discussion, Conclusions

I expect to see various patterns of "diluting" concentrations and/or "flushing" raising concentrations, during storm events. Additionally, there will be different patterns of hysteresis between the storms, with some showing clockwise loops, others anticlockwise, and many more complex figure-8s and complex patterns. The hypothesis is that these patterns, both flushing vs. diluting and hysteresis, will be influenced by and show correlations with storm size, seasonality, and sub-catchment-level land-use.

### References

Evans, Christopher, and Trevor D. Davies. “Causes of Concentration/Discharge Hysteresis and Its Potential as a Tool for Analysis of Episode Hydrochemistry.” Water Resources Research 34, no. 1 (1998): 129–37. https://doi.org/10.1029/97WR01881.

Galloway, James N., John D. Aber, Jan Willem Erisman, et al. “The Nitrogen Cascade.” BioScience 53, no. 4 (2003): 341. https://doi.org/10.1641/0006-3568(2003)053%255B0341:TNC%255D2.0.CO;2.

Smith, V.H., G.D. Tilman, and J.C. Nekola. “Eutrophication: Impacts of Excess Nutrient Inputs on Freshwater, Marine, and Terrestrial Ecosystems.” Environmental Pollution 100, nos. 1–3 (1999): 179–96. https://doi.org/10.1016/S0269-7491(99)00091-3.

Vaughan, M. C. H., W. B. Bowden, J. B. Shanley, et al. “High‐frequency Dissolved Organic Carbon and Nitrate Measurements Reveal Differences in Storm Hysteresis and Loading in Relation to Land Cover and Seasonality.” Water Resources Research 53, no. 7 (2017): 5345–63. https://doi.org/10.1002/2017WR020491.

Whitehead, P. G., R. L. Wilby, R. W. Battarbee, M. Kernan, and A. J. Wade. “A Review of the Potential Impacts of Climate Change on Surface Water Quality.” Hydrological Sciences Journal 54, no. 1 (2009): 101–23. https://doi.org/10.1623/hysj.54.1.101.

Zuecco, G., D. Penna, M. Borga, and H. J. Van Meerveld. “A Versatile Index to Characterize Hysteresis between Hydrological Variables at the Runoff Event Timescale.” Hydrological Processes 30, no. 9 (2016): 1449–66. https://doi.org/10.1002/hyp.10681.